# Big Data Project Notebook

This notebook contains the solution for the project proposed for the Big Data Course in the year 2021/22

In [1]:
!pip install pandas -U

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pandas in /d/hpc/home/mf4782/.local/lib/python3.8/site-packages (1.4.2)


In [2]:
#All the necessary imports
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import dask_jobqueue
import pandas as pd
import dask.dataframe as dd
from distributed import Client
from contextlib import suppress 
import platform; print( platform.python_version () )

3.8.5


## Reading Data

First we will read data through Dask and SLURM 

In [3]:
path = "/d/hpc/projects/FRI/bigdata/data/NYTickets"
files = os.listdir(path)
files = list([f for f in files if ".csv" in f])
print(files)

['2019.csv', '2020.csv', '2017.csv', '2018.csv', '2016.csv', '2021.csv', '2015.csv', '2014.csv', '2022.csv']


In [4]:
with suppress(Exception):
    client.shutdown()

cluster = dask_jobqueue.SLURMCluster(
            queue = 'all', 
            processes=2,
            cores=16, 
            memory='8GB',
            scheduler_options={'dashboard_address': ':21722'},
            death_timeout=120 # seconds
          )

client = Client(cluster, timeout="120s")
display(client.cluster)

/cvmfs/sling.si/modules/el7/software/Anaconda3/2020.11/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 21722 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45417 instead
  warnings.warn(


In [5]:
client.cluster.scale(16)
display(client.cluster)

In [7]:
col2type = {'Summons Number': pd.Int64Dtype(),
 'Plate ID': 'str',
 'Registration State': 'str',
 'Plate Type': 'str',
 'Issue Date': 'str', # mogoce str
 'Violation Code': pd.Int64Dtype(),
 'Vehicle Body Type': 'str',
 'Vehicle Make': 'str',
 'Issuing Agency': 'str',
 'Street Code1': pd.Int64Dtype(),
 'Street Code2': pd.Int64Dtype(),
 'Street Code3': pd.Int64Dtype(),
 'Vehicle Expiration Date': 'str', # TODO
 'Violation Location': pd.Float64Dtype(),
 'Violation Precinct': pd.Int64Dtype(),
 'Issuer Precinct': pd.Int64Dtype(),
 'Issuer Code': pd.Int64Dtype(),
 'Issuer Command': 'str',
 'Issuer Squad': 'str',
 'Violation Time': 'str', # TODO
 'Time First Observed': 'str', # TODO
 'Violation County': 'str',
 'Violation In Front Of Or Opposite': 'str',
 'Number': 'str', # ni int
 'Street': 'str',
 'Intersecting Street': 'str',
 'Date First Observed': 'str', # todo
 'Law Section': 'str',
 'Sub Division': 'str',
 'Violation Legal Code': 'str', # TODO
 'Days Parking In Effect    ': 'str',
 'From Hours In Effect': 'str',
 'To Hours In Effect': 'str',
 'Vehicle Color': 'str',
 'Unregistered Vehicle?': 'str', # TODO
 'Vehicle Year': pd.Int64Dtype(),
 'Meter Number': 'str',
 'Feet From Curb': pd.Int64Dtype(),
 'Violation Post Code': 'str',
 'Violation Description': 'str',
 'No Standing or Stopping Violation':'str', # todo
 'Hydrant Violation': 'str', # todo
 'Double Parking Violation': 'str', # todo
 'House Number': 'str',
 'Street Name': 'str'}
 #'Days Parking In Effect': 'str'}
def read_messages_into_df_dask(path):
    df = dd.read_csv(path, parse_dates=['Issue Date', 'Violation Time'],dtype=col2type)
    return df

multiple_dfs = [read_messages_into_df_dask(path+"/"+f) for f in files]

df = dd.concat(multiple_dfs)

TypeError: Cannot interpret 'Float64Dtype()' as a data type

In [9]:
csv_2014 = dd.read_csv(path + "/2014.csv", blocksize="4GB", assume_missing=True)

In [12]:
a = csv_2014.compute()

KilledWorker: ("('read-csv-ccb89af7667de31107ca4dae3fce30db', 0)", <Worker 'tcp://153.5.72.161:46111', name: SLURMCluster-7-1, memory: 0, processing: 1>)